## Calibration experiment for fatigue estimation parameters


In [1]:
import sys
import pytrigno
import Plot2D
from time import perf_counter
from scipy import signal
import os
import reprlib
import numpy as np
from numpy import arange
import PyQt5
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
import socket
import struct
import math 


### This experiment will consist on two parts: 
- A first part in order to record the maximum voluntary contraction (MVC). Value that is going to be used for normalysing the emg signals.
- A second part will consist on exerting 0.3 times the MVC for as long as possible. With this the fatigue parameters are going to be calculated.

## First part: 

The user will place the sensor on the muscle of interest, the anterior deltoid. Once it is properly located, the experiment will begin. For this first part the subject will exert as much force as possible with such muscle against the blocking hands of the experimenter. The pose of the subject will remain a 125º shoulder flexion with derotated scapula, in sitting position, erect posture and without back support. An example of this can be found in the following image. The subject will be asked to exert that force for 3 seconds.

<center><img src='calib.png'></center> 


#### The LE function will calculate the linear envelope of the EMG signal. For this, only butterworth filters are used.
 
 
 
**Parameters:** 
- f: EMG signal
- hpf: Cutoff frequency in Hz for the high pass filter
- lpf: Cutoff frequency in Hz for the low pass filter
- hpn: order of the high pass butterworth filter
- lpn: order of the low pass butterworth filter
- fs: sampling frequency of the signal

**Output:** 
- sle: linear envelope of the EMG signal. With the same shape as the 

In [2]:

def LE(f, hpf, lpf, hpn, lpn, fs):

    nyq = fs/2
    hpfn = hpf/nyq
    lpfn = lpf/nyq

    b, a = signal.butter(hpn, hpfn, btype='highpass', analog=False, output='ba' )
    f = signal.filtfilt(b, a, f)

    f = np.absolute(f)

    d, c = signal.butter(lpn, lpfn, btype='lowpass', analog=False, output='ba' )
    sle = signal.filtfilt(d, c, f)

    return sle


In [3]:
dev = pytrigno.TrignoEMG(channel_range=(0, 0), samples_per_read=300, host='localhost', units='mV')
i = 0
j = 0
k = 0
dev.start()

d1 = [0]
d2 = [0]
d3 = [0]

In [4]:
def test1():
    
    global p1, l1, f1, s1, d1, i, timerout1
    
     
        
    if (i == 0):
        timerout1.start(5000)
            
    t = np.arange(0,300,1)
    f1 = dev.read()
    assert f1.shape == (1, 300)

    f1 = f1.T
    s1 = f1.reshape(300,)
    p1.trace("EMG", t, s1)

    l1 = LE(s1, 20, 50, 6, 4, 2000)
    p1.trace("LE", t, l1)
        
    d1.append(l1.max())
    i = 1


### Trial 1.1


In [5]:
p1 = Plot2D.Plot2D()
timer1 = QtCore.QTimer()
timerout1 = QtCore.QTimer()
timer1.timeout.connect(test1)
timerout1.timeout.connect(lambda: timer1.stop())
timer1.start()
p1.start()
 

In [6]:
def test2():
        
    global p2, l2, f2, s2, d2, j, timerout2
        
    if (j == 0):
        timerout2.start(5000)
            
    t = np.arange(0,300,1)
    f2 = dev.read()
    assert f2.shape == (1, 300)

    f2 = f2.T
    s2 = f2.reshape(300,)
    p2.trace("EMG", t, s2)

    l2 = LE(s2, 20, 50, 6, 4, 2000)
    p2.trace("LE", t, l2)
        
    d2.append(l2.max())
    j = 1

### Trial 1.2

In [7]:
j = 0
p2 = Plot2D.Plot2D()
timer2 = QtCore.QTimer()
timerout2 = QtCore.QTimer()
timer2.timeout.connect(test2)
timerout2.timeout.connect(lambda: timer2.stop())
timer2.start()
p2.start()

    

In [8]:
def test3():
    
    global p3, l3, f3, s3, d3, k, timerout3
        
    if (k == 0):
        timerout3.start(5000)
            
    t = np.arange(0,300,1)
    f3 = dev.read()
    assert f3.shape == (1, 300)

    f3 = f3.T
    s3 = f3.reshape(300,)
    p3.trace("EMG", t, s3)

    l3 = LE(s3, 20, 50, 6, 4, 2000)
    p3.trace("LE", t, l3)
        
    d3.append(l3.max())
    k = 1


### Trial 1.3

In [9]:

p3 = Plot2D.Plot2D()
timer3 = QtCore.QTimer()
timerout3 = QtCore.QTimer()
timer3.timeout.connect(test3)
timerout3.timeout.connect(lambda: timer3.stop())
timer3.start()
p3.start()

    

#### Calculated MVC:

In [10]:
dev.stop()

d1 = np.array(d1)
d2 = np.array(d2)
d3 = np.array(d3)

dtot = [d1.max(), d2.max(), d3.max()]
MVC = sum(dtot)/3
print('MVC', MVC, sep=' = ')

MVC = 0.8337518250470621


## Second Part

In this second part the subject will repeat the task, but this time the goal will be to keep the muscle activation, represented by the red line, between the two thresholds that are drown in the visual feedback plot (green and blue line). When fatigue is accumulated enough for use to loose control over the red line and it crosses the yelow line it will stop and save the time we endured. We can see a screenshot of the graph below. This part will consist on two trials. 

<center><img src='plot.png'></center> 

In [11]:
d4 = [0]
z = 0
th = 0.1
t = np.arange(0, 300, 1)
mvc = np.full(len(t), 0.3*MVC)
hth = np.full (len(t), (0.3*MVC)+th)
lth = np.full (len(t), (0.3*MVC)-th)
#dev2 = pytrigno.TrignoEMG(channel_range=(0, 0), samples_per_read=300, host='localhost', units='mV')
#dev2.start()

In [12]:
def test4():
    
    global p4, l4, f4, s4, d4, z, hth, lth, MVC, mvc, Tend1, tstart, th, t
    
    if (z == 0):
        tstart = perf_counter()
    
    z += 1
    
    p4.trace("0.3MVC", t, mvc)
    p4.trace("High th", t, hth)
    p4.trace("Low th", t, lth)
    
    f4 = dev.read()
    assert f4.shape == (1, 300)
    
    f4 = f4.T
    s4 = f4.reshape(300,)
    l4 = LE(s4, 20, 50, 6, 4, 2000)
    l4 = np.full(len(t), l4.max())
    p4.trace("LE", t, l4)
    
#    if (l4.all() > (0.3*MVC)+0.1):
#        tstop = perf_counter()
#        print("The experiment is wrong")
#        z = 0
#        timer4.stop()
    
    if (l4.max() < (0.3*MVC)-th and z > 100):
        tstop= perf_counter()
        Tend1 = tstop-tstart
        print (Tend1)
        timer4.stop()
        dev.stop()      

### Trial 2.1

In [13]:
p4 = Plot2D.Plot2D()
timer4 = QtCore.QTimer()
timer4.timeout.connect(test4)
dev.start()
timer4.start()
p4.start()

12.842549399999996


In [14]:
def test5():
    
    global p5, l5, f5, s5, d5, z, hth, lth, MVC, mvc, Tend2, tstart2, th, t
    
    if (z == 0):
        tstart2 = perf_counter()
    
    z += 1
    
    p5.trace("0.3MVC", t, mvc)
    p5.trace("High th", t, hth)
    p5.trace("Low th", t, lth)
    
    f5 = dev.read()
    assert f5.shape == (1, 300)
    
    f5 = f5.T
    s5 = f5.reshape(300,)
    l5 = LE(s5, 20, 50, 6, 4, 2000)
    l5 = np.full(len(t), l5.max())
    p5.trace("LE", t, l5)
    
#    if (l4.all() > (0.3*MVC)+0.1):
#        tstop = perf_counter()
#        print("The experiment is wrong")
#        z = 0
#        timer4.stop()
    
    if (l5.max() < (0.3*MVC)-th and z > 100):
        tstop2= perf_counter()
        Tend2 = tstop2 - tstart2
        print (Tend2)
        timer5.stop()
        dev.stop() 

### Trial 2.2

In [15]:
z = 0
p5 = Plot2D.Plot2D()
timer5 = QtCore.QTimer()
timer5.timeout.connect(test5)
dev.start()
timer5.start()
p5.start()



20.8973546


#### Calculated Tend

In [16]:
Tend = (Tend1 + Tend2)/2
print("Tend", Tend, sep = ' = ')

file = open("calibparam.txt", "w")
mvcs = repr(MVC)
tends = repr(Tend)

file.write(mvcs + "\n" + tends)
file.close()


Tend = 16.869951999999998
